In [1]:
import sys
import os
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
from logger_config import logger

from typing import Annotated, Sequence, List, Literal 
from pydantic import BaseModel, Field 
from langchain_core.messages import HumanMessage
from langchain_community.tools.tavily_search import TavilySearchResults 
from langgraph.types import Command 
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import create_react_agent 
from IPython.display import Image, display 
from dotenv import load_dotenv
from langchain_experimental.tools import PythonREPLTool
import pprint

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
llm = ChatGroq(model="llama-3.1-8b-instant")

In [3]:
tavily_search = TavilySearchResults(max_results=1)

python_repl_tool = PythonREPLTool()

In [4]:
python_repl_tool.invoke("x = 5; print(x)")

Python REPL can execute arbitrary code. Use with caution.


'5\n'

In [5]:
class Supervisor(BaseModel):
    next: Literal["enhancer", "researcher", "coder"] = Field(
        description="Determines which specialist to activate next in the workflow sequence: "
                    "'enhancer' when user input requires clarification, expansion, or refinement, "
                    "'researcher' when additional facts, context, or data collection is necessary, "
                    "'coder' when implementation, computation, or technical problem-solving is required."
    )
    reason: str = Field(
        description="Detailed justification for the routing decision, explaining the rationale behind selecting the particular specialist and how this advances the task toward completion."
    )

def supervisor_node(state: MessagesState) -> Command[Literal["enhancer", "researcher", "coder"]]:

    system_prompt = ('''
                 
        You are a workflow supervisor managing a team of three specialized agents: Prompt Enhancer, Researcher, and Coder. Your role is to orchestrate the workflow by selecting the most appropriate next agent based on the current state and needs of the task. Provide a clear, concise rationale for each decision to ensure transparency in your decision-making process.

        **Team Members**:
        1. **Prompt Enhancer**: Always consider this agent first. They clarify ambiguous requests, improve poorly defined queries, and ensure the task is well-structured before deeper processing begins.
        2. **Researcher**: Specializes in information gathering, fact-finding, and collecting relevant data needed to address the user's request.
        3. **Coder**: Focuses on technical implementation, calculations, data analysis, algorithm development, and coding solutions.

        **Your Responsibilities**:
        1. Analyze each user request and agent response for completeness, accuracy, and relevance.
        2. Route the task to the most appropriate agent at each decision point.
        3. Maintain workflow momentum by avoiding redundant agent assignments.
        4. Continue the process until the user's request is fully and satisfactorily resolved.

        Your objective is to create an efficient workflow that leverages each agent's strengths while minimizing unnecessary steps, ultimately delivering complete and accurate solutions to user requests.
                 
    ''')
    
    messages = [
        {"role": "system", "content": system_prompt},  
    ] + state["messages"] 

    response = llm.with_structured_output(Supervisor).invoke(messages)

    goto = response.next
    reason = response.reason

    logger.info(f"--- Workflow Transition: Supervisor → {goto.upper()} ---")
    
    return Command(
        update={
            "messages": [
                HumanMessage(content=reason, name="supervisor")
            ]
        },
        goto=goto,  
    )


In [6]:
def enhancer_node(state: MessagesState) -> Command[Literal["supervisor"]]:

    """
        Enhancer agent node that improves and clarifies user queries.
        Takes the original user input and transforms it into a more precise,
        actionable request before passing it to the supervisor.
    """
   
    system_prompt = (
        "You are a Query Refinement Specialist with expertise in transforming vague requests into precise instructions. Your responsibilities include:\n\n"
        "1. Analyzing the original query to identify key intent and requirements\n"
        "2. Resolving any ambiguities without requesting additional user input\n"
        "3. Expanding underdeveloped aspects of the query with reasonable assumptions\n"
        "4. Restructuring the query for clarity and actionability\n"
        "5. Ensuring all technical terminology is properly defined in context\n\n"
        "Important: Never ask questions back to the user. Instead, make informed assumptions and create the most comprehensive version of their request possible."
    )

    messages = [
        {"role": "system", "content": system_prompt},  
    ] + state["messages"]  

    enhanced_query = llm.invoke(messages)

    logger.info(f"--- Workflow Transition: Prompt Enhancer → Supervisor ---")

    return Command(
        update={
            "messages": [  
                HumanMessage(
                    content=enhanced_query.content, 
                    name="enhancer"  
                )
            ]
        },
        goto="supervisor", 
    )

In [7]:
def research_node(state: MessagesState) -> Command[Literal["validator"]]:

    """
        Research agent node that gathers information using Tavily search.
        Takes the current task state, performs relevant research,
        and returns findings for validation.
    """
    
    research_agent = create_react_agent(
        llm,  
        tools=[tavily_search],  
        state_modifier= "You are an Information Specialist with expertise in comprehensive research. Your responsibilities include:\n\n"
            "1. Identifying key information needs based on the query context\n"
            "2. Gathering relevant, accurate, and up-to-date information from reliable sources\n"
            "3. Organizing findings in a structured, easily digestible format\n"
            "4. Citing sources when possible to establish credibility\n"
            "5. Focusing exclusively on information gathering - avoid analysis or implementation\n\n"
            "Provide thorough, factual responses without speculation where information is unavailable."
    )

    result = research_agent.invoke(state)

    logger.info(f"--- Workflow Transition: Researcher → Validator ---")

    return Command(
        update={
            "messages": [ 
                HumanMessage(
                    content=result["messages"][-1].content,  
                    name="researcher"  
                )
            ]
        },
        goto="validator", 
    )


In [8]:
def code_node(state: MessagesState) -> Command[Literal["validator"]]:

    code_agent = create_react_agent(
        llm,
        tools=[python_repl_tool],
        state_modifier=(
            "You are a coder and analyst. Focus on mathematical calculations, analyzing, solving math questions, "
            "and executing code. Handle technical problem-solving and data tasks."
        )
    )

    result = code_agent.invoke(state)

    logger.info(f"--- Workflow Transition: Coder → Validator ---")

    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="coder")
            ]
        },
        goto="validator",
    )


In [9]:
# System prompt providing clear instructions to the validator agent
system_prompt = '''
    Your task is to ensure reasonable quality. 
    Specifically, you must:
    - Review the user's question (the first message in the workflow).
    - Review the answer (the last message in the workflow).
    - If the answer addresses the core intent of the question, even if not perfectly, signal to end the workflow with 'FINISH'.
    - Only route back to the supervisor if the answer is completely off-topic, harmful, or fundamentally misunderstands the question.
    
    - Accept answers that are "good enough" rather than perfect
    - Prioritize workflow completion over perfect responses
    - Give benefit of doubt to borderline answers
    
    Routing Guidelines:
    1. 'supervisor' Agent: ONLY for responses that are completely incorrect or off-topic.
    2. Respond with 'FINISH' in all other cases to end the workflow.
'''

class Validator(BaseModel):
    next: Literal["supervisor", "FINISH"] = Field(
        description="Specifies the next worker in the pipeline: 'supervisor' to continue or 'FINISH' to terminate."
    )
    reason: str = Field(
        description="The reason for the decision."
    )

def validator_node(state: MessagesState) -> Command[Literal["supervisor", "__end__"]]:

    user_question = state["messages"][0].content
    agent_answer = state["messages"][-1].content

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_question},
        {"role": "assistant", "content": agent_answer},
    ]

    response = llm.with_structured_output(Validator).invoke(messages)

    goto = response.next
    reason = response.reason

    if goto == "FINISH" or goto == END:
        goto = END  
        logger.info(" --- Transitioning to END ---")  
    else:
        logger.info(f"--- Workflow Transition: Validator → Supervisor ---")
 

    return Command(
        update={
            "messages": [
                HumanMessage(content=reason, name="validator")
            ]
        },
        goto=goto, 
    )


In [10]:
graph = StateGraph(MessagesState)

graph.add_node("supervisor", supervisor_node) 
graph.add_node("enhancer", enhancer_node)  
graph.add_node("researcher", research_node) 
graph.add_node("coder", code_node) 
graph.add_node("validator", validator_node)  

graph.add_edge(START, "supervisor")  
app = graph.compile()

In [11]:
logger.info(app.get_graph().draw_mermaid())

2025-05-19 15:55:51 | 3451679309.py | <module> | Line: 1 | INFO | %%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	supervisor(supervisor)
	enhancer(enhancer)
	researcher(researcher)
	coder(coder)
	validator(validator)
	__end__([<p>__end__</p>]):::last
	__start__ --> supervisor;
	supervisor -.-> enhancer;
	supervisor -.-> researcher;
	supervisor -.-> coder;
	enhancer -.-> supervisor;
	researcher -.-> validator;
	coder -.-> validator;
	validator -.-> supervisor;
	validator -.-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [12]:
inputs = {
    "messages": [
        ("user", "Weather in Chennai"),
    ]
}

for event in app.stream(inputs):
    for key, value in event.items():
        if value is None:
            continue
        last_message = value.get("messages", [])[-1] if "messages" in value else None
        if last_message:
            pprint.pprint(f"Output from node '{key}':")
            pprint.pprint(last_message, indent=2, width=80, depth=None)
            logger.info("")


2025-05-19 15:49:35 | 2193137005.py | supervisor_node | Line: 42 | INFO | --- Workflow Transition: Supervisor → RESEARCHER ---
2025-05-19 15:49:35 | 3656673417.py | <module> | Line: 15 | INFO | 


"Output from node 'supervisor':"
HumanMessage(content='The user query is about weather, which requires factual information and data collection. The researcher agent is best suited to gather the relevant data needed to provide an accurate answer.', additional_kwargs={}, response_metadata={}, name='supervisor', id='2950b2e0-22ba-4d2f-b9fc-7fcb8fa69cbf')


2025-05-19 15:49:39 | 2625631429.py | research_node | Line: 23 | INFO | --- Workflow Transition: Researcher → Validator ---
2025-05-19 15:49:39 | 3656673417.py | <module> | Line: 15 | INFO | 


"Output from node 'researcher':"
HumanMessage(content='The current weather in Chennai is patchy light rain with thunder. The temperature is 33.4°C or 92.1°F, and the humidity is 67%. The wind speed is 13.3 km/h or 8.3 mph from the east-southeast direction. The atmospheric pressure is 1001.0 mb or 29.56 in, and the visibility is 5.0 km or 3.0 miles. The UV index is 3.3.', additional_kwargs={}, response_metadata={}, name='researcher', id='a94706d6-b96d-422e-b027-6500665428d0')


2025-05-19 15:49:40 | 1262067928.py | validator_node | Line: 45 | INFO |  --- Transitioning to END ---
2025-05-19 15:49:40 | 3656673417.py | <module> | Line: 15 | INFO | 


"Output from node 'validator':"
HumanMessage(content='Answered the user query about Chennai weather.', additional_kwargs={}, response_metadata={}, name='validator', id='f1c6e9a6-8208-4154-8719-905f9cc399b5')


In [12]:
inputs = {
    "messages": [
        ("user", "Give me the 20th fibonacci number"),
    ]
}
for event in app.stream(inputs):
    for key, value in event.items():
        if value is None:
            continue
        pprint.pprint(f"Output from node '{key}':")
        pprint.pprint(value, indent=2, width=80, depth=None)
        logger.info("")


2025-05-19 15:55:55 | 2193137005.py | supervisor_node | Line: 42 | INFO | --- Workflow Transition: Supervisor → RESEARCHER ---
2025-05-19 15:55:55 | 1400057110.py | <module> | Line: 12 | INFO | 


"Output from node 'supervisor':"
{ 'messages': [ HumanMessage(content='To gather required data and resources for calculating the 20th Fibonacci number.', additional_kwargs={}, response_metadata={}, name='supervisor', id='19104510-806e-42fc-88e2-0ceb71d349ae')]}


2025-05-19 15:56:07 | 2625631429.py | research_node | Line: 23 | INFO | --- Workflow Transition: Researcher → Validator ---
2025-05-19 15:56:07 | 1400057110.py | <module> | Line: 12 | INFO | 


"Output from node 'researcher':"
{ 'messages': [ HumanMessage(content='The 20th Fibonacci number is 6,765.', additional_kwargs={}, response_metadata={}, name='researcher', id='2da834d1-eabb-4584-8375-e14290133054')]}


2025-05-19 15:56:07 | 1262067928.py | validator_node | Line: 45 | INFO |  --- Transitioning to END ---
2025-05-19 15:56:07 | 1400057110.py | <module> | Line: 12 | INFO | 


"Output from node 'validator':"
{ 'messages': [ HumanMessage(content='Provided the 20th Fibonacci number directly.', additional_kwargs={}, response_metadata={}, name='validator', id='db7f1c18-05a5-4af7-bb02-b9b66bb94724')]}
